In [15]:
%load_ext autoreload
%autoreload 2

In [44]:
import pandas as pd
import re

from utils import get_data, Config

## Load Data + Preprocessing

In [321]:
# Load all data
initial_data = """
SELECT "Result".*, 
"Response".raw_header, "Response".status_code, "Response".label, "Response".resp_type,
"Browser".name, "Browser".version, "Browser".headless_mode, "Browser".os, "Browser".automation_mode, "Browser".add_info
FROM "Result"
JOIN "Response" ON "Result".response_id = "Response".id JOIN "Browser" ON "Result".browser_id = "Browser".id
WHERE "Response".created_at <= DATE('2023-11-21') and "Browser".name != 'Unknown';
"""
df = get_data(Config(), initial_data)

Connecting to the PostgreSQL database...
Connection successful


In [322]:
# Extra columns
df["outcome_str"] = df["outcome_value"].fillna("None").astype(str)

# Unify outcomes that are the same
# Firefox: {'error': 'object "TypeError: NetworkError when attempting to fetch resource."', 'headers': ''} Chromium: {'error': 'object "TypeError: Failed to fetch"', 'headers': ''}
# Safari: {'error': 'object "TypeError: Load failed"', 'headers': ''}
df["outcome_str"] = df["outcome_str"].replace("{'error': 'object \"TypeError: NetworkError when attempting to fetch resource.\"', 'headers': ''}", "{'error': 'object \"TypeError: Failed to fetch\"', 'headers': ''}")
df["outcome_str"] = df["outcome_str"].replace("{'error': 'object \"TypeError: Load failed\"', 'headers': ''}", "{'error': 'object \"TypeError: Failed to fetch\"', 'headers': ''}")

# For document referrer we do not want to know the exact resp_id and count
# Maybe we even only want to know whether it is a origin or the full URl?
df['outcome_str'] = df['outcome_str'].replace(r'resp_id=\d+', 'resp_id=<resp_id>', regex=True)
df['outcome_str'] = df['outcome_str'].replace(r'count=\d+', 'count=<count>', regex=True)

In [323]:
df["outcome_str"].unique()

array(["{'error': 'null', 'headers': 'access-control-allow-orgin,content-length,date,server,test,'}",
       '{\'error\': \'object "TypeError: Failed to fetch"\', \'headers\': \'\'}',
       "{'error': 'null', 'headers': 'access-control-allow-credentials,content-length,date,server,test,'}",
       "{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-orgin,content-length,date,server,test,'}",
       'message send',
       "{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-orgin,access-control-expose-headers,content-length,date,server,test,'}",
       "{'error': 'null', 'headers': 'cache-control,content-type,'}",
       'message timeout', 'fullscreenEnabled: true',
       'fullscreenEnabled: false', 'window.originAgentCluster: false',
       'window.originAgentCluster: true', "{'requestStart != 0': True}",
       "{'requestStart != 0': False}",
       'document.refe

## Overview

In [324]:
display(df.head(2))
display(df.describe())
display(df.columns)

,id,outcome_type,outcome_value,test_name,test_status,test_message,test_stack,org_scheme,org_host,resp_scheme,...,status_code,label,resp_type,name,version,headless_mode,os,automation_mode,add_info,outcome_str
0,1,<class 'dict'>,"{'error': 'null', 'headers': 'access-control-a...",fetch_GET,0,None,None,http,sub.headers.websec.saarland,http,...,200,CORS,basic,chrome,119,headless-new,Ubuntu 22.04,selenium,,"{'error': 'null', 'headers': 'access-control-a..."
1,2,<class 'dict'>,"{'error': 'null', 'headers': 'access-control-a...",fetch_TEST,0,None,None,http,sub.headers.websec.saarland,http,...,200,CORS,basic,chrome,119,headless-new,Ubuntu 22.04,selenium,,"{'error': 'null', 'headers': 'access-control-a..."


,id,test_status,browser_id,testcase_id,response_id,created_at,updated_at,status_code
count,198664.000000,198664.000000,198664.000000,198664.0,198664.000000,198664,198664,198664.000000
mean,99332.500000,0.000705,55.105263,1.0,78.027927,2023-11-21 12:45:18.583738112,2023-11-21 12:45:18.583738368,215.998470
min,1.000000,0.000000,5.000000,1.0,3.000000,2023-11-21 10:34:47.097638,2023-11-21 10:34:47.097641,200.000000
25%,49666.750000,0.000000,13.000000,1.0,23.000000,2023-11-21 11:38:47.412738560,2023-11-21 11:38:47.412740352,200.000000
50%,99332.500000,0.000000,60.000000,1.0,46.000000,2023-11-21 12:46:22.775672064,2023-11-21 12:46:22.775673088,200.000000
75%,148998.250000,0.000000,75.000000,1.0,151.000000,2023-11-21 13:50:22.174202368,2023-11-21 13:50:22.174202368,200.000000
max,198664.000000,2.000000,116.000000,1.0,217.000000,2023-11-21 15:13:12.751466,2023-11-21 15:13:12.751467,302.000000
std,57349.501277,0.037536,40.144439,0.0,66.518630,NaN,NaN,37.093123


Index(['id', 'outcome_type', 'outcome_value', 'test_name', 'test_status',
       'test_message', 'test_stack', 'org_scheme', 'org_host', 'resp_scheme',
       'resp_host', 'relation_info', 'browser_id', 'testcase_id',
       'response_id', 'status', 'full_url', 'created_at', 'updated_at',
       'raw_header', 'status_code', 'label', 'resp_type', 'name', 'version',
       'headless_mode', 'os', 'automation_mode', 'add_info', 'outcome_str'],
      dtype='object')

In [325]:
# Collected results per browser
df.groupby(["name", "version", "os", "headless_mode", "automation_mode"])["id"].count().sort_values(ascending=False).to_frame()

id
name    version                   os           headless_mode automation_mode       
brave   1.59.120 (118.0.5993.88)  Ubuntu 22.04 headless-new  selenium         10456
                                               xvfb          selenium         10456
        1.60.114 (119.0.6045.124) Ubuntu 22.04 headless-new  selenium         10456
                                               xvfb          selenium         10456
        1.60.118 (119.0.6045.163) macOS 14.0   headless-new  selenium         10456
                                               real          selenium         10456
chrome  119                       Ubuntu 22.04 headless-new  selenium         10456
                                               xvfb          selenium         10456
                                  macOS 14.0   headless-new  selenium         10456
                                               real          selenium         10456
edge    119                       Ubuntu 22.04 headless-new  selenium         10456
                                               xvfb          selenium         10456
                                  macOS 14.0   headless-new  selenium         10456
                                               real          selenium         10456
firefox 119                       Ubuntu 22.04 headless      selenium         10456
                                               xvfb          selenium         10456
                                  macOS 14.0   headless      selenium         10456
                                               real          selenium         10456
safari  17.0                      macOS 14.0   real          selenium         10456

In [326]:
# Timeouts
display(df.groupby(["name", "version", "os", "headless_mode", "automation_mode"])["test_status"].value_counts().to_frame())

display(df.loc[df["test_status"] == 2])

count
name    version                   os           headless_mode automation_mode test_status       
brave   1.59.120 (118.0.5993.88)  Ubuntu 22.04 headless-new  selenium        0            10456
                                               xvfb          selenium        0            10456
        1.60.114 (119.0.6045.124) Ubuntu 22.04 headless-new  selenium        0            10456
                                               xvfb          selenium        0            10456
        1.60.118 (119.0.6045.163) macOS 14.0   headless-new  selenium        0            10456
                                               real          selenium        0            10456
chrome  119                       Ubuntu 22.04 headless-new  selenium        0            10456
                                               xvfb          selenium        0            10456
                                  macOS 14.0   headless-new  selenium        0            10456
                                               real          selenium        0            10456
edge    119                       Ubuntu 22.04 headless-new  selenium        0            10456
                                               xvfb          selenium        0            10455
                                                                             2                1
                                  macOS 14.0   headless-new  selenium        0            10455
                                                                             2                1
                                               real          selenium        0            10456
firefox 119                       Ubuntu 22.04 headless      selenium        0            10456
                                               xvfb          selenium        0            10456
                                  macOS 14.0   headless      selenium        0            10456
                                               real          selenium        0            10456
safari  17.0                      macOS 14.0   real          selenium        0            10388
                                                                             2               68

,id,outcome_type,outcome_value,test_name,test_status,test_message,test_stack,org_scheme,org_host,resp_scheme,...,status_code,label,resp_type,name,version,headless_mode,os,automation_mode,add_info,outcome_str
51278,51391,<class 'NoneType'>,None,referrer_iframe,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,200,RP,basic,safari,17.0,real,macOS 14.0,selenium,,None
51282,51395,<class 'NoneType'>,None,referrer_iframe,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,200,RP,basic,safari,17.0,real,macOS 14.0,selenium,,None
51444,51399,<class 'NoneType'>,None,referrer_iframe,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,200,RP,basic,safari,17.0,real,macOS 14.0,selenium,,None
52107,52079,<class 'NoneType'>,None,referrer_iframe,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,200,RP,basic,safari,17.0,real,macOS 14.0,selenium,,None
52111,52083,<class 'NoneType'>,None,referrer_iframe,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,200,RP,basic,safari,17.0,real,macOS 14.0,selenium,,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62734,62678,<class 'NoneType'>,None,referrer_iframe,2,Test timed out,None,https,sub.headers.websec.saarland,http,...,302,RP,basic,safari,17.0,real,macOS 14.0,selenium,,None
62735,62679,<class 'NoneType'>,None,referrer_iframe,2,Test timed out,None,https,sub.headers.websec.saarland,https,...,302,RP,basic,safari,17.0,real,macOS 14.0,selenium,,None
62736,62680,<class 'NoneType'>,None,referrer_iframe,2,Test timed out,None,https,sub.headers.websec.saarland,https,...,302,RP,basic,safari,17.0,real,macOS 14.0,selenium,,None
153193,153240,<class 'NoneType'>,None,perfAPI_img,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,200,TAO,basic,edge,119,headless-new,macOS 14.0,selenium,,None


In [327]:
#TODO: easy way to redo timed out tests!

In [328]:
# Time taken!
df.groupby(["name", "version", "os", "headless_mode", "automation_mode", "org_scheme"])["created_at"].agg(["min", "max"]).apply(lambda x: x["max"] - x["min"], axis=1).to_frame()

0
name    version                   os           headless_mode automation_mode org_scheme                       
brave   1.59.120 (118.0.5993.88)  Ubuntu 22.04 headless-new  selenium        http       0 days 00:15:52.729060
                                                                             https      0 days 00:13:28.093020
                                               xvfb          selenium        http       0 days 00:15:53.539157
                                                                             https      0 days 00:13:33.120399
        1.60.114 (119.0.6045.124) Ubuntu 22.04 headless-new  selenium        http       0 days 00:15:50.330800
                                                                             https      0 days 00:13:29.664335
                                               xvfb          selenium        http       0 days 00:16:01.039583
                                                                             https      0 days 00:13:30.205549
        1.60.118 (119.0.6045.163) macOS 14.0   headless-new  selenium        http       0 days 00:16:00.851072
                                                                             https      0 days 00:13:17.676180
                                               real          selenium        http       0 days 00:16:47.333179
                                                                             https      0 days 00:13:51.071158
chrome  119                       Ubuntu 22.04 headless-new  selenium        http       0 days 00:15:32.276187
                                                                             https      0 days 00:13:11.597734
                                               xvfb          selenium        http       0 days 00:15:32.517923
                                                                             https      0 days 00:13:14.107244
                                  macOS 14.0   headless-new  selenium        http       0 days 00:10:32.869022
                                                                             https      0 days 00:13:15.386574
                                               real          selenium        http       0 days 00:11:05.393402
                                                                             https      0 days 00:13:43.572152
edge    119                       Ubuntu 22.04 headless-new  selenium        http       0 days 00:10:37.168142
                                                                             https      0 days 00:13:43.430620
                                               xvfb          selenium        http       0 days 00:10:29.766958
                                                                             https      0 days 00:13:35.611712
                                  macOS 14.0   headless-new  selenium        http       0 days 00:10:54.594401
                                                                             https      0 days 00:13:29.270315
                                               real          selenium        http       0 days 00:11:14.907553
                                                                             https      0 days 00:13:58.131905
firefox 119                       Ubuntu 22.04 headless      selenium        http       0 days 00:11:46.705398
                                                                             https      0 days 00:14:26.595717
                                               xvfb          selenium        http       0 days 00:12:03.017871
                                                                             https      0 days 00:14:44.644456
                                  macOS 14.0   headless      selenium        http       0 days 00:10:44.408936
                                                                             https      0 days 00:14:12.761382
                                               real          selenium        http       0 days 00:10:48.618109
  

## Response analysis

- We have 10456 unique tests: (test_name x relation_info) x org_scheme x (resp_scheme x resp_host) x response_id)
- They should all have 10 (num browsers in various settings) outcomes
    -  Due to the HTTPS Upgrade bug many have either 16 or 4 outcomes only (as the 6 http outcomes of Chrome + Brave instead resulted in HTTPS outcomes)
    -  Without org_scheme they should have 20; a couple tests have less than 20 (due to the Firefox errors and/or timeouts?)

In [329]:
# Unique tests
tests = {
    "Fetch": [4, 2, 8, 6], 
    "Framing": [9, 2, 8, 35], # 7+11+17
    "PP": [4, 2, 8, 24],
    "OAC": [3, 2, 8, 14],
    "TAO": [1, 2, 8, 13],
    "RP": [2, 2, 8, 16],
    "CSP-SCRIPT": [2, 2, 8, 17],
    "COEP": [2, 2, 8, 11],
    "CORP": [4, 2, 8, 11],
    "CSP-IMG": [1, 2, 8, 17],
    "HSTS": [2, 1, 2, 14],
    "COOP": [1, 2, 8, 11],
}
tests = pd.DataFrame.from_dict(tests, columns=["tests", "org_origins", "resp_origins", "responses"], orient="index")
tests["All tests"] = tests.prod(axis=1)
tests.loc["Sum"] = tests.sum(axis=0)
tests

,tests,org_origins,resp_origins,responses,All tests
Fetch,4,2,8,6,384
Framing,9,2,8,35,5040
PP,4,2,8,24,1536
OAC,3,2,8,14,672
TAO,1,2,8,13,208
RP,2,2,8,16,512
CSP-SCRIPT,2,2,8,17,544
COEP,2,2,8,11,352
CORP,4,2,8,11,704
CSP-IMG,1,2,8,17,272


In [330]:
# SELECT count(*) as c2, count(DISTINCT browser_id), count(DISTINCT outcome_value) as c1, test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id 
# FROM "Result" GROUP by test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id order by c1 DESC, c2 DESC

res = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])[["browser_id", "outcome_str"]].agg(["count", "nunique"])
display(res)
display(res.value_counts().to_frame())

browser_id  \
                                                                                                                                    count   
test_name             org_scheme org_host                    resp_scheme resp_host                   relation_info response_id              
accesswindow_direct   http       sub.headers.websec.saarland http        headers.webappsec.eu        direct        70                  19   
                                                                                                                   71                  19   
                                                                                                                   72                  19   
                                                                                                                   73                  19   
                                                                                                                   74                  19   
...                                                                                                                                   ...   
upgradeHSTS_subdomain http       sub.headers.websec.saarland https       sub.headers.websec.saarland subdomain     138                 19   
                                                                                                                   139                 19   
                                                                                                                   140                 19   
                                                                                                                   141                 19   
                                                                                                                   142                 19   

                                                                                                                                        \
                                                                                                                               nunique   
test_name             org_scheme org_host                    resp_scheme resp_host                   relation_info response_id           
accesswindow_direct   http       sub.headers.websec.saarland http        headers.webappsec.eu        direct        70               19   
                                                                                                                   71               19   
                                                                                                                   72               19   
                                                                                                                   73               19   
                                                                                                                   74               19   
...                                                                                                                                ...   
upgradeHSTS_subdomain http       sub.headers.websec.saarland https       sub.headers.websec.saarland subdomain     138              19   
                                                                                                                   139              19   
                                                                                                                   140              19   
                                                                                                                   141              19   
                                                                                                                   142              19   

                                                                                                                               outcome_str  \
                                                                             

count
(browser_id, count) (browser_id, nunique) (outcome_str, count) (outcome_str, nunique)       
19                  19                    19                   1                        8279
                                                               2                        1878
                                                               3                         294
                                                               4                           5

### Differences:
- COOP (accesswindow_direct):
    - [ ] Edge + Firefox: redirect first
    - [ ] Brave + Chrome: COOP first?
-  Framing:
    -  [ ] framing_iframe_sandbox: CSP-FA *; Firefox allows, all other browsers deny?
- PP not supported in Firefox
- OAC not supported in Firefox
    - [ ] Also random/caching behavior?
- PerfAPI:
    - [ ] Chrome/Firefox have different rules when to create an entry at all?
- COEP (CORP):
    - [ ] Firefox incorrect behavior for same-origin (applies same-site?)
- CORP:
    - [ ] same-origin: Edge does not check the scheme?
    - [x] Firefox applies CORP on object tags (even though it should not!)
    - [ ] Some other CORP and/or COEP strange stuff (Firefox)
- Referrer Policy: different default in Brave (unsafe-url is never allowed)
- Subresource-loading (CSP): differences due to HTTPS Upgrades
- Script Execution (CSP):
    - [ ] Safari sandbox bug?! (maybe related https://bugs.webkit.org/show_bug.cgi?id=223848 unsure though)
- Upgrade (HSTS):
    - [ ] Safari bug?

In [331]:
def get_uniques(df):
    browsers = f"{df['name'].unique().tolist()}"
    try:
        outcome = unique_outcomes[df.name[0]]
    except KeyError:
        outcome = {}
        #outcome = []
    outcome[browsers] = df.name[1]
    #outcome.append(f"{df.name[1]} ({browsers})")
    unique_outcomes[df.name[0]] = outcome

for grouping, group in df.groupby(["test_name", "relation_info"]):
    
    unique_outcomes = {}
    group.groupby(["name"])["outcome_str"].value_counts(normalize=True).reset_index().groupby(["outcome_str", "proportion"], group_keys=True).apply(get_uniques)
    d = pd.DataFrame.from_dict(unique_outcomes, orient="index")
    if d.shape[1] != 1:
        print(grouping)
        # display(unique_outcomes)
        display(group.groupby(["name"])["outcome_str"].value_counts(normalize=True).to_frame().unstack(0))
        for value, dic in unique_outcomes.items():
            values = [f"{v} ({k})" for k,v in dic.items()]
            # print(f"{value}: {values}")
        display(d)
        print()


('accesswindow_direct', 'direct')


proportion            \
name                                                    brave    chrome   
outcome_str                                                               
{'window.open.opener': 'null'}                       0.340909  0.261364   
{'window.open.opener': 'object "[object Window]"'}   0.659091  0.738636   

                                                                        \
name                                                    edge   firefox   
outcome_str                                                              
{'window.open.opener': 'null'}                      0.181818  0.181818   
{'window.open.opener': 'object "[object Window]"'}  0.818182  0.818182   

                                                              
name                                                  safari  
outcome_str                                                   
{'window.open.opener': 'null'}                      0.181818  
{'window.open.opener': 'object "[object Window]"'}  0.818182

,"['edge', 'firefox', 'safari']",['chrome'],['brave']
{'window.open.opener': 'null'},0.181818,0.261364,0.340909
"{'window.open.opener': 'object ""[object Window]""'}",0.818182,0.738636,0.659091



('framing_embed', 'direct')


proportion                                     
name                 brave    chrome      edge   firefox safari
outcome_str                                                    
message send      0.332143  0.332143  0.332143  0.332143    NaN
message timeout   0.667857  0.667857  0.667857  0.667857    1.0

...

proportion                              
name                         brave chrome  edge   firefox safari
outcome_str                                                     
error                         0.25   0.25  0.25  0.431818    NaN
load                          0.75   0.75  0.75  0.568182    NaN
timeout: no event fired        NaN    NaN   NaN       NaN    1.0

,"['brave', 'chrome', 'edge']",['firefox'],['safari']
error,0.25,0.431818,NaN
load,0.75,0.568182,NaN
timeout: no event fired,NaN,NaN,1.0



('upgradeHSTS_direct', 'direct')


proportion                                \
name                                brave    chrome      edge   firefox   
outcome_str                                                               
{'response.redirected': False}   0.821429  0.821429  0.821429  0.821429   
{'response.redirected': True}    0.178571  0.178571  0.178571  0.178571   

                                          
name                              safari  
outcome_str                               
{'response.redirected': False}  0.785714  
{'response.redirected': True}   0.214286

,['safari'],"['brave', 'chrome', 'edge', 'firefox']"
{'response.redirected': False},0.785714,0.821429
{'response.redirected': True},0.214286,0.178571


In [334]:
def make_clickable(url):
    # Clickable links for debugging; set browser_id=2 (unknown) to not count them in our real data collection
    url = re.sub(r"browser_id=(\d+)", "browser_id=2", url)
    return f'<a href="{url}" target="_blank">{url}</a>'

test_name_seen = set()
for test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id in res.loc[res[("outcome_str", "nunique")] != 1].index:
    # Only look at each feature group once
    if test_name in test_name_seen:
        continue
    test_name_seen.add(test_name)
    print(test_name)
    rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"] == response_id)]
    with pd.option_context("display.max_colwidth", 200):
        #display(rows[["outcome_str", "name", "headless_mode", "os", "version"]].value_counts().to_frame())
        display(rows[["outcome_str", "name"]].value_counts().to_frame())
        disp = rows.drop_duplicates(subset="outcome_str")[["browser_id", "headless_mode", "name", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
        disp = disp.style.format({'full_url': make_clickable})
        display(disp)
        
    

accesswindow_direct


count
outcome_str                                        name          
{'window.open.opener': 'null'}                     brave        6
{'window.open.opener': 'object "[object Window]"'} edge         4
                                                   firefox      4
{'window.open.opener': 'null'}                     chrome       2
{'window.open.opener': 'object "[object Window]"'} chrome       2
                                                   safari       1

,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
8245,13,headless-new,chrome,{'window.open.opener': 'null'},"[['Cross-Origin-Opener-Policy', 'same-origin-allow-popups']]",accesswindow_direct,http,sub.headers.websec.saarland,http,direct,headers.webappsec.eu,http://sub.headers.websec.saarland/_hp/tests/window-references-coop.sub.html?resp_type=basic&browser_id=2&label=COOP&first_id=71&last_id=71&scheme=http
13276,5,real,chrome,"{'window.open.opener': 'object ""[object Window]""'}","[['Cross-Origin-Opener-Policy', 'same-origin-allow-popups']]",accesswindow_direct,http,sub.headers.websec.saarland,http,direct,headers.webappsec.eu,http://sub.headers.websec.saarland/_hp/tests/window-references-coop.sub.html?resp_type=basic&browser_id=2&label=COOP&first_id=71&last_id=71&scheme=http


framing_embed


count
outcome_str     name          
message send    brave        6
                chrome       4
                edge         4
                firefox      4
message timeout safari       1

...

count
outcome_str             name          
load                    brave        6
                        chrome       4
                        edge         4
                        firefox      4
timeout: no event fired safari       1

,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
6617,13,headless-new,chrome,load,"[['Cross-Origin-Resource-Policy', 'unsafe-none']]",subresourceloadingCORP_object,http,sub.headers.websec.saarland,http,direct,headers.webappsec.eu,http://sub.headers.websec.saarland/_hp/tests/subresource-loading-corp.sub.html?resp_type=basic&browser_id=2&label=CORP&first_id=44&last_id=44&scheme=http
54381,7,real,safari,timeout: no event fired,"[['Cross-Origin-Resource-Policy', 'unsafe-none']]",subresourceloadingCORP_object,http,sub.headers.websec.saarland,http,direct,headers.webappsec.eu,http://sub.headers.websec.saarland/_hp/tests/subresource-loading-corp.sub.html?resp_type=basic&browser_id=2&label=CORP&first_id=44&last_id=44&scheme=http


upgradeHSTS_direct


count
outcome_str                    name          
{'response.redirected': False} brave        6
                               chrome       4
                               edge         4
                               firefox      4
{'response.redirected': True}  safari       1

,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
9260,13,headless-new,chrome,{'response.redirected': False},"[['Strict-Transport-Security', 'max-age=20, max-age=20; includeSubDomains, includeSubDomains, , max-age=20; includeSubDomains; preload, max-age=0, max-age=-5']]",upgradeHSTS_direct,http,sub.headers.websec.saarland,https,direct,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=2&label=HSTS&first_id=139&last_id=139&scheme=http
55979,7,real,safari,{'response.redirected': True},"[['Strict-Transport-Security', 'max-age=20, max-age=20; includeSubDomains, includeSubDomains, , max-age=20; includeSubDomains; preload, max-age=0, max-age=-5']]",upgradeHSTS_direct,http,sub.headers.websec.saarland,https,direct,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=2&label=HSTS&first_id=139&last_id=139&scheme=http


# Crash analysis

### Exceptions thrown
- Fixed:
    - Only Firefox; both headless + Xvfb
        - OAC + RP
        - Both on HTTP and HTTPS
        - Response_ids: 185, 186, 341, 342; these are the ones that have status code 302!
        - Reason seems to be that Firefox switches to another tab in these cases and thus the wait for crashes. Solution: forcefully switch back to the original tab before waiting!
    - Only Brave: both headless + Xvfb
        - OAC + RP
        - Both on HTTP and HTTPS
        - Reason seems to be the tab switching, if deactivated no issue occurs

In [147]:
with open("/home/ubuntu/wpt/_hp/tools/crawler/desktop-selenium-2023-11-20 16:09:24.649008.log", "r") as f:
    crashes = []
    for l in f.readlines():
        if "649008: http" in l:
            match = re.match(".*(http|https)://.*browser_id=(\d+)&label=(.*)&first_id=(\d+).*", l)
            crashes.append([match[1], match[2], match[3], match[4]])

In [148]:
crashes = pd.DataFrame(crashes, columns=["scheme", "browser_id", "label", "response_id"]).drop_duplicates()
crashes

,scheme,browser_id,label,response_id
0,http,59,RP,172
2,http,59,RP,173
4,https,59,OAC,339
6,https,59,RP,181
8,http,75,RP,171
10,http,75,RP,172
12,https,75,OAC,330
14,https,75,OAC,338
16,https,75,RP,180
18,https,74,OAC,337
